In [1]:
#!pip install nltk
#!pip install torchtext

In [2]:
#!pip uninstall torchtext -y


In [3]:
#!pip install torchtext==0.17.2

In [4]:
import numpy as np
import math

import torch
import torch.nn as nn
from torch import Tensor

from nltk.tokenize import word_tokenize
from nltk import *
import nltk
from torchtext.vocab import build_vocab_from_iterator

![example of positional encoding](./images/img_positional_encoding.webp)

In [5]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
sequences = ["I wonder what will come next",
             "This is a basic example paragraph",
             "Hello, what is a basic split?"]

In [7]:
tokenized_sequences = []
# 구두점 정의
punctuations = [',', '?', '!', '.']
for seq in sequences:
    tokens = word_tokenize(seq)
    filtered_tokens = [token for token in tokens if token not in punctuations]
    tokenized_sequences.append(filtered_tokens)

In [8]:
tokenized_sequences

[['I', 'wonder', 'what', 'will', 'come', 'next'],
 ['This', 'is', 'a', 'basic', 'example', 'paragraph'],
 ['Hello', 'what', 'is', 'a', 'basic', 'split']]

In [18]:
# set the output to 2 decimal places without scientific notation
torch.set_printoptions(precision=2, sci_mode=False)
vocab = build_vocab_from_iterator(tokenized_sequences)
stoi = vocab.get_stoi()
for token, index in stoi.items():
    print(f"Token: {token}, Index: {index}")

Token: a, Index: 0
Token: This, Index: 6
Token: wonder, Index: 13
Token: next, Index: 9
Token: basic, Index: 1
Token: is, Index: 2
Token: what, Index: 3
Token: Hello, Index: 4
Token: paragraph, Index: 10
Token: I, Index: 5
Token: example, Index: 8
Token: split, Index: 11
Token: come, Index: 7
Token: will, Index: 12


vocab.get_stoi()는 {"단어": 인덱스} 형태의 dict 임. 

``` python
stoi = vocab.get_stoi()
for token, index in stoi.items():
    print(f"Token: {token}, Index: {index}")
```

In [14]:
stoi = vocab.get_stoi()
# index the sequences 
indexed_sequences = [[stoi[word] for word in seq] for seq in tokenized_sequences]

In [15]:
indexed_sequences

[[5, 13, 3, 12, 7, 9], [6, 2, 0, 1, 8, 10], [4, 3, 2, 0, 1, 11]]

In [17]:
vocab_size

14

In [16]:
# convert the sequences to a tensor
tensor_sequences = torch.tensor(indexed_sequences).long()

# vocab size
vocab_size = len(stoi)

# embedding dimensions
d_model = 4

# create the embeddings
lut = nn.Embedding(vocab_size, d_model) # look-up table (lut)

# embed the sequence
embeddings = lut(tensor_sequences)

embeddings

tensor([[[     0.53,     -0.97,     -2.19,      0.71],
         [     0.52,      0.85,     -0.07,     -0.54],
         [    -1.08,      0.13,     -0.13,      0.97],
         [     0.40,      0.64,      0.17,     -0.86],
         [    -0.99,      0.66,      0.55,     -1.31],
         [     0.60,      0.55,     -0.57,      0.44]],

        [[    -0.37,      0.42,     -0.30,     -0.33],
         [    -0.07,     -0.86,      0.32,     -0.19],
         [    -1.62,     -1.25,      1.27,      0.24],
         [     1.33,      1.08,      0.00,     -0.50],
         [    -1.11,     -1.39,      1.09,     -0.17],
         [     0.35,      0.57,      1.54,      0.88]],

        [[     0.28,     -1.27,      0.10,      0.73],
         [    -1.08,      0.13,     -0.13,      0.97],
         [    -0.07,     -0.86,      0.32,     -0.19],
         [    -1.62,     -1.25,      1.27,      0.24],
         [     1.33,      1.08,      0.00,     -0.50],
         [    -0.72,      1.23,     -1.48,     -0.09]]],
    

In [20]:
embeddings.shape


torch.Size([3, 6, 4])

In [19]:
tensor_sequences.shape

torch.Size([3, 6])

다음 단계는 각 시퀀스 내 각 단어의 위치를 위치 인코딩(positional encoding)을 통해 인코딩하는 것입니다.  
아래 함수는 위 정의를 따릅니다. 언급할 만한 유일한 변화는 **𝐿** 이 **_max_length_** 로 표기된다는 점입니다.  
이는 거의 모든 시퀀스를 적절히 인코딩할 수 있도록 보장하기 위해 보통 수천 단위의 매우 큰 값으로  
설정됩니다.   
이를 통해 동일한 위치 인코딩 행렬을 다양한 길이의 시퀀스에 사용할 수 있으며, 추가 전에 적절한 길이로  
잘라낼 수 있습니다.

![positional formula](./images/pe_formula.webp)

In [13]:
def gen_pe(max_length, d_model, n):

  # generate an empty matrix for the positional encodings (pe)
  pe = np.zeros(max_length*d_model).reshape(max_length, d_model) 

  # for each position
  for k in np.arange(max_length):

    # for each dimension
    for i in np.arange(d_model//2):

      # calculate the internal value for sin and cos
      theta = k / (n ** ((2*i)/d_model))       

      # even dims: sin   
      pe[k, 2*i] = math.sin(theta) 

      # odd dims: cos               
      pe[k, 2*i+1] = math.cos(theta)

  return pe

# maximum sequence length
max_length = 10
n = 1000
encodings = gen_pe(max_length, d_model, n)

The output of the encoding contains 10 position encoding vectors.

In [14]:
encodings

array([[ 0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.84147098,  0.54030231,  0.03161751,  0.99950004],
       [ 0.90929743, -0.41614684,  0.0632034 ,  0.99800067],
       [ 0.14112001, -0.9899925 ,  0.09472609,  0.99550337],
       [-0.7568025 , -0.65364362,  0.12615407,  0.99201066],
       [-0.95892427,  0.28366219,  0.1574559 ,  0.98752602],
       [-0.2794155 ,  0.96017029,  0.18860029,  0.98205394],
       [ 0.6569866 ,  0.75390225,  0.21955609,  0.97559988],
       [ 0.98935825, -0.14550003,  0.25029236,  0.9681703 ],
       [ 0.41211849, -0.91113026,  0.28077835,  0.95977264]])

앞서 언급했듯이, **_max_length_** 는 10으로 설정됩니다.  
이는 필요한 길이보다 크지만, 길이가 7, 8, 9 또는 10인 다른 시퀀스가  
있을 경우 동일한 위치 인코딩 행렬을 사용할 수 있도록 보장하기 위함입니다.   
단지 적절한 길이로 잘라내면 됩니다.  
아래에서는 임베딩의 시퀀스 길이가 6이므로 인코딩을 이에 맞게 잘라낼 수 있습니다.

In [15]:
# select the first six tokens
seq_length = embeddings.shape[1]
encodings[:seq_length]

array([[ 0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.84147098,  0.54030231,  0.03161751,  0.99950004],
       [ 0.90929743, -0.41614684,  0.0632034 ,  0.99800067],
       [ 0.14112001, -0.9899925 ,  0.09472609,  0.99550337],
       [-0.7568025 , -0.65364362,  0.12615407,  0.99201066],
       [-0.95892427,  0.28366219,  0.1574559 ,  0.98752602]])

모든 시퀀스의 길이가 동일하므로 하나의 위치 인코딩 행렬만 필요하며,  
이를 PyTorch를 사용하여 세 시퀀스에 모두 브로드캐스트할 수 있습니다.   
이 예제에서 임베딩된 배치의 형태는 (3,6,4)이고, 위치 인코딩은 잘라내기  
전에는 (10,4) 형태를 가지며, 잘라낸 후에는 (6,4) 형태가 됩니다.  
이 행렬은 이후 (3,6,4) 인코딩 행렬을 생성하기 위해 브로드캐스트됩니다  
(이미지에서 확인 가능).   

브로드캐스트에 대한 자세한 내용은 A Simple Introduction to Broadcasting을 참조하세요.  
이 과정 덕분에 두 행렬을 문제없이 더할 수 있습니다.  

When the positional encodings are added to the embeddings,  
the output is the same as the image at the beginning of the section.

In [16]:
embeddings + torch.tensor(encodings[:seq_length]) # encodings[:6]

tensor([[[ 0.90,  0.29,  1.73,  0.89],
         [ 1.65, -0.96,  1.77,  1.36],
         [ 0.92, -0.20, -1.15,  1.43],
         [ 0.11, -0.73, -0.24,  0.53],
         [-1.85,  0.07,  1.75,  0.91],
         [-1.98, -1.33, -1.04,  0.40]],

        [[ 0.13,  0.79,  0.03,  1.79],
         [-0.03,  0.41,  0.03,  2.61],
         [ 1.54, -0.98,  0.38,  1.74],
         [ 0.77, -1.34, -0.80,  0.90],
         [-0.59, -1.53, -0.90,  1.71],
         [-0.79, -0.43, -0.77,  0.39]],

        [[ 0.37,  1.16, -1.41,  2.23],
         [ 0.85,  0.76, -1.18,  1.43],
         [ 0.03, -0.55,  0.07,  2.61],
         [ 0.77, -1.55,  0.41,  1.74],
         [-0.13, -1.01, -0.77,  0.90],
         [-0.85,  0.06, -0.18,  0.27]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

이 출력은 모델의 다음 계층인 **멀티헤드 어텐션(Multi-head Attention)**으로 전달됩니다.   
멀티헤드 어텐션은 다음 기사에서 다룰 예정입니다.  

하지만 이 기본 구현은 중첩 루프(nested loop)를 사용하기 때문에, 특히 $d_{model}$ 과   
**_max_length_** 값이 클 경우 비효율적입니다.   
대신, PyTorch 중심의 더 효율적인 접근 방식을 사용할 수 있습니다.

![](./images/formula_2.gif)

PyTorch의 기능을 활용하기 위해, 원래의 수식,   
특히 분모 부분을 로그 규칙을 사용하여 수정해야 합니다.

분모는 다음과 같습니다:

$\Large \frac{1}{n^\frac{2i}{d_model}}$

분모를 수정하기 위해 𝑛의 지수를 음수로 만들어 분자로 이동시킵니다.    
그런 다음, 규칙 7을 사용하여 전체 수식을 𝑒의 지수로 변환합니다. 이후,   
규칙 3을 사용하여 지수를 로그(log) 바깥으로 꺼냅니다.   
이를 간소화하면 최종 결과를 얻을 수 있습니다.

$\Large \frac{1}{n^\frac{2i}{d_model}} = n^{-\frac{2i}{d_{model}}} = e^{log{(n^{-\frac{2i}{d_{model}}})}} = e^{-\frac{2i\ log(n)}{d_{model}}} $


이는 위치 인코딩의 모든 분모를 한 번에 생성하는 데 사용할 수 있기 때문에 중요합니다.  
아래에서 알 수 있듯이, 4차원 임베딩의 경우 필요한 분모는 두 개뿐입니다.  
이는 𝑖가 차원을 나타낼 때, 분모가 2i마다 한 번씩만 변하기 때문입니다.   
이러한 패턴은 각 위치에서 반복됩니다:  

![](./images/pe_formula_3.webp)

Since only the highest number that i can be set to is d_model divided by 2,   
the terms can be calculated once:

In [17]:
d_model = 4
n = 100

div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(n) / d_model))

이 짧은 코드 조각은 필요한 모든 분모를 생성하는 데 사용할 수 있습니다.  
이 예제에서는 d_{model}이 4로 설정되었고, n은 100으로 설정되었습니다.  
출력 결과는 두 개의 분모입니다:

In [18]:
div_term

tensor([1.00, 0.10])

여기서부터는 PyTorch의 인덱싱 기능을 활용하여 몇 줄의 코드로 
전체 위치 인코딩 행렬을 생성할 수 있습니다. 다음 단계는 
𝑘 부터 L−1까지 각 위치를 생성하는 것입니다.

In [19]:
max_length = 10

# generate the positions into a column matrix
k = torch.arange(0, max_length).unsqueeze(1) 
print(k)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])


위치와 분모가 준비되었으므로, 사인(sin) 및 코사인(cos) 함수의 내부 값을  
쉽게 계산할 수 있습니다.

![](./images/pe_formula_4.webp)

k와 div_term을 곱하면 모든 위치에 대한 입력값을 계산할 수 있습니다.   
PyTorch는 행렬을 자동으로 브로드캐스트하여 곱셈을 수행합니다.   
이 경우, 대응하는 요소끼리 곱해지는 **아다마르 곱(Hadamard product)** 이며,  
행렬 곱셈이 아님을 유의하세요:

![](./images/pe_formula_5.webp)

In [20]:
k*div_term

tensor([[0.00, 0.00],
        [1.00, 0.10],
        [2.00, 0.20],
        [3.00, 0.30],
        [4.00, 0.40],
        [5.00, 0.50],
        [6.00, 0.60],
        [7.00, 0.70],
        [8.00, 0.80],
        [9.00, 0.90]])

이 계산의 출력 결과는 위 이미지에서 확인할 수 있습니다.   
이제 남은 작업은 입력값을 사인(sin)과 코사인(cos) 함수에 넣고,   
이를 적절히 행렬에 저장하는 것입니다.  

이를 시작하려면 적절한 크기의 빈 행렬을 생성하면 됩니다:

In [21]:
# generate an empty tensor
pe = torch.zeros(max_length, d_model)

print(pe)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


이제 짝수 열(사인 값을 나타냄)은 pe[:, 0::2]를 사용하여 선택할 수 있습니다.  
이는 PyTorch에게 모든 행과 짝수 열을 선택하라는 의미입니다.   
동일하게 홀수 열(코사인 값을 나타냄)은 pe[:, 1::2]를 사용하여 선택할 수 있습니다.   
이는 PyTorch에게 모든 행과 홀수 열을 선택하라는 의미입니다.  
k×div_term의 결과는 필요한 모든 입력값을 저장하고 있으므로,   
이를 사용하여 각 짝수 및 홀수 열을 계산할 수 있습니다.

In [22]:
# set the odd values (columns 1 and 3)
pe[:, 0::2] = torch.sin(k * div_term)

# set the even values (columns 2 and 4)
pe[:, 1::2] = torch.cos(k * div_term)
     
# add a dimension for broadcasting across sequences: optional       
pe = pe.unsqueeze(0)

print(pe)

tensor([[[ 0.00,  1.00,  0.00,  1.00],
         [ 0.84,  0.54,  0.10,  1.00],
         [ 0.91, -0.42,  0.20,  0.98],
         [ 0.14, -0.99,  0.30,  0.96],
         [-0.76, -0.65,  0.39,  0.92],
         [-0.96,  0.28,  0.48,  0.88],
         [-0.28,  0.96,  0.56,  0.83],
         [ 0.66,  0.75,  0.64,  0.76],
         [ 0.99, -0.15,  0.72,  0.70],
         [ 0.41, -0.91,  0.78,  0.62]]])


이 값들은 중첩 for-루프를 사용하여 얻은 값과 동일합니다.   
요약하자면, 아래는 모든 코드를 하나로 정리한 것입니다

In [23]:
max_length = 10
d_model = 4
n = 100

def gen_pe(max_length, d_model, n):
  # calculate the div_term
  div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(n) / d_model))

  # generate the positions into a column matrix
  k = torch.arange(0, max_length).unsqueeze(1)

  # generate an empty tensor
  pe = torch.zeros(max_length, d_model)

  # set the even values
  pe[:, 0::2] = torch.sin(k * div_term)

  # set the odd values
  pe[:, 1::2] = torch.cos(k * div_term)

  # add a dimension       
  pe = pe.unsqueeze(0)        

  # the output has a shape of (1, max_length, d_model)
  return pe                           

gen_pe(max_length, d_model, n)  

tensor([[[ 0.00,  1.00,  0.00,  1.00],
         [ 0.84,  0.54,  0.10,  1.00],
         [ 0.91, -0.42,  0.20,  0.98],
         [ 0.14, -0.99,  0.30,  0.96],
         [-0.76, -0.65,  0.39,  0.92],
         [-0.96,  0.28,  0.48,  0.88],
         [-0.28,  0.96,  0.56,  0.83],
         [ 0.66,  0.75,  0.64,  0.76],
         [ 0.99, -0.15,  0.72,  0.70],
         [ 0.41, -0.91,  0.78,  0.62]]])

더 복잡하긴 하지만, PyTorch는 향상된 성능을 제공하기 때문에   
이 구현 방식을 머신 러닝에 사용합니다.

**Positional Encoding in Transformers**

이제 복잡한 작업이 끝났으므로 구현은 비교적 간단합니다.   
이 구현은 The Annotated Transformer와 PyTorch에서 파생되었습니다.  
참고로 n의 기본값은 10,000이며, max_length의 기본값은 5,000입니다.  

이 구현은 또한 **드롭아웃(dropout)** 을 포함합니다.   
드롭아웃은 주어진 확률p에 따라 입력 요소 일부를 랜덤하게 0으로 설정합니다.     
이는 정규화(regularization)에 도움을 주며, 뉴런들이 서로 과도하게   
의존(co-adapting)하는 것을 방지합니다.  
출력값은 또한 $\frac{1}{1-p}$로 스케일링됩니다.  
이 기사에서 드롭아웃에 대해 깊이 다루지는 않으니, 자세한 내용은   
**드롭아웃 레이어(Dropout Layer)** 에 관한 기사를 참조하세요.  

트랜스포머 모델의 나머지 부분으로 넘어가기 전에 드롭아웃에   
익숙해지는 것이 중요합니다.   
드롭아웃은 거의 모든 다른 계층에서도 사용되기 때문입니다.

In [24]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, dropout: float = 0.1, max_length: int = 5000):
    """
    Args:
      d_model:      dimension of embeddings
      dropout:      randomly zeroes-out some of the input
      max_length:   max sequence length
    """
    # inherit from Module
    super().__init__()     

    # initialize dropout                  
    self.dropout = nn.Dropout(p=dropout)      

    # create tensor of 0s
    pe = torch.zeros(max_length, d_model)    

    # create position column   
    k = torch.arange(0, max_length).unsqueeze(1)  

    # calc divisor for positional encoding 
    div_term = torch.exp(                                 
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
    )

    # calc sine on even indices
    pe[:, 0::2] = torch.sin(k * div_term)    

    # calc cosine on odd indices   
    pe[:, 1::2] = torch.cos(k * div_term)  

    # add dimension     
    pe = pe.unsqueeze(0)          

    # buffers are saved in state_dict but not trained by the optimizer                        
    self.register_buffer("pe", pe)                        

  def forward(self, x: Tensor):
    """
    Args:
      x:        embeddings (batch_size, seq_length, d_model)
    
    Returns:
                embeddings + positional encodings (batch_size, seq_length, d_model)
    """
    # add positional encoding to the embeddings
    x = x + self.pe[:, : x.size(1)].requires_grad_(False) 

    # perform dropout
    return self.dropout(x)

**Forward Pass**

To perform the forward pass, the same embedded sequences from earlier can be used.

In [25]:
embeddings

tensor([[[ 0.90, -0.71,  1.73, -0.11],
         [ 0.81, -1.50,  1.74,  0.36],
         [ 0.01,  0.22, -1.21,  0.43],
         [-0.03,  0.26, -0.34, -0.47],
         [-1.09,  0.73,  1.63, -0.09],
         [-1.02, -1.61, -1.20, -0.59]],

        [[ 0.13, -0.21,  0.03,  0.79],
         [-0.88, -0.13,  0.00,  1.61],
         [ 0.63, -0.56,  0.31,  0.74],
         [ 0.63, -0.35, -0.90, -0.10],
         [ 0.17, -0.87, -1.02,  0.72],
         [ 0.17, -0.72, -0.93, -0.60]],

        [[ 0.37,  0.16, -1.41,  1.23],
         [ 0.01,  0.22, -1.21,  0.43],
         [-0.88, -0.13,  0.00,  1.61],
         [ 0.63, -0.56,  0.31,  0.74],
         [ 0.63, -0.35, -0.90, -0.10],
         [ 0.11, -0.22, -0.33, -0.71]]], grad_fn=<EmbeddingBackward0>)

시퀀스가 임베딩된 후, 위치 인코딩 행렬을 생성할 수 있습니다.   
드롭아웃은 임베딩과 위치 인코딩 간의 합산을 쉽게 확인할 수 있도록 0.0으로 설정되었습니다.  
값이 처음부터 구현한 값과 다른 이유는 n의 기본값이 100이 아닌 10,000으로 설정되었기 때문입니다.

In [26]:
d_model = 4
max_length = 10
dropout = 0.0

# create the positional encoding matrix
pe = PositionalEncoding(d_model, dropout, max_length)

# preview the values
pe.state_dict()

OrderedDict([('pe',
              tensor([[[ 0.00,  1.00,  0.00,  1.00],
                       [ 0.84,  0.54,  0.01,  1.00],
                       [ 0.91, -0.42,  0.02,  1.00],
                       [ 0.14, -0.99,  0.03,  1.00],
                       [-0.76, -0.65,  0.04,  1.00],
                       [-0.96,  0.28,  0.05,  1.00],
                       [-0.28,  0.96,  0.06,  1.00],
                       [ 0.66,  0.75,  0.07,  1.00],
                       [ 0.99, -0.15,  0.08,  1.00],
                       [ 0.41, -0.91,  0.09,  1.00]]]))])

합산하기 전에 시퀀스의 형태는 (batch_size,seq_length,d_model), 즉 (3,6,4)입니다.  
위치 인코딩도 잘라내고 브로드캐스트된 후 동일한 크기를 가지므로,   
순방향 전달(forward pass)의 출력 크기도 (batch_size,seq_length,d_model),    
즉 (3,6,4)로 유지됩니다. 이는 4차원 공간에 임베딩된 6개의 토큰으로 이루어진   
3개의 시퀀스를 나타내며, 위치 인코딩을 통해 각 토큰의 시퀀스 내 위치를 표시합니다.

In [27]:
pe(embeddings)

tensor([[[ 0.90,  0.29,  1.73,  0.89],
         [ 1.65, -0.96,  1.75,  1.36],
         [ 0.92, -0.20, -1.19,  1.43],
         [ 0.11, -0.73, -0.31,  0.53],
         [-1.85,  0.07,  1.67,  0.91],
         [-1.98, -1.33, -1.15,  0.41]],

        [[ 0.13,  0.79,  0.03,  1.79],
         [-0.03,  0.41,  0.01,  2.61],
         [ 1.54, -0.98,  0.33,  1.74],
         [ 0.77, -1.34, -0.87,  0.90],
         [-0.59, -1.53, -0.98,  1.72],
         [-0.79, -0.43, -0.88,  0.40]],

        [[ 0.37,  1.16, -1.41,  2.23],
         [ 0.85,  0.76, -1.20,  1.43],
         [ 0.03, -0.55,  0.02,  2.61],
         [ 0.77, -1.55,  0.34,  1.74],
         [-0.13, -1.01, -0.86,  0.90],
         [-0.85,  0.06, -0.28,  0.28]]], grad_fn=<AddBackward0>)